<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
   
                                          
                                          
                                          
                                          
                                          
#                                                 Regression with Abalone Data Set: Elastic Net Model




#                                                        Gladys Murage

#                              College of Business, Engineering, and  Technology, National University

#                                         DDS8555 v1: PREDICTIVE ANALYSIS(3602869492)

#                                                        Dr MOHAMED NABEEL

#                                                            March 17, 2025


<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>


## Step 1: Import necessary libraries

In [60]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline  # Import Pipeline


## Step 2: Load the Abalone dataset
The Abalone dataset is often used for regression tasks where the goal is to predict the age of abalone based on physical measurements.
### Loading Data:

The dataset is now loaded from train.csv and test.csv files.

The sample_submission.csv file is loaded to create a submission file with predictions.

### Handling Categorical Data:

Since the dataset contains a categorical column like Sex, it is converted to numerical values using a mapping.

In [61]:
# Load the datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Debug: Check lengths
print(f"Length of test_data: {len(test_data)}")
print(f"Length of sample_submission: {len(sample_submission)}")

# Convert categorical 'Sex' column to numerical values
if 'Sex' in train_data.columns:
    train_data['Sex'] = train_data['Sex'].map({'M': 0, 'F': 1, 'I': 2})
    test_data['Sex'] = test_data['Sex'].map({'M': 0, 'F': 1, 'I': 2})

# Separate features and target
X = train_data.drop("Rings", axis=1)  # Features
y = train_data["Rings"]  # Target

# Ensure target values are non-negative
if (y < 0).any():
    raise ValueError("Target values (y) contain negative values. RMSLE cannot be used.")


Length of test_data: 60411
Length of sample_submission: 60411


## Step 3: Split the dataset into training and testing sets

In [62]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 4: Standardize the features
Elastic Net is sensitive to the scale of the input features, so it's a good practice to standardize them.

In [63]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(test_data)  # Standardize the test data as well

## Step 5: Train the Elastic Net model

In [64]:

# Initialize the Elastic Net model
elastic_net = ElasticNet(random_state=42)

# Train the model
elastic_net.fit(X_train, y_train)

ElasticNet(random_state=42)

## Step 6: Make predictions and evaluate the model

In [65]:


# Make predictions on the validation set
y_val_pred = elastic_net.predict(X_val)


# Evaluate the model on the validation set using RMSLE
rmsle = np.sqrt(mean_squared_log_error(y_val, y_val_pred))
print(f"Root Mean Squared Logarithmic Error on Validation Set: {rmsle}")

# Debug: Check length of predictions
print(f"Length of predictions (y_test_pred): {len(y_test_pred)}")


Root Mean Squared Logarithmic Error on Validation Set: 0.20534378215404459
Length of predictions (y_test_pred): 60411


# Step 7: Tuning Hyperparameters
Now I will tune the alpha and l1_ratio parameters using techniques like Grid Search or Randomized Search to find the best combination.
## 1. Imports and Initialization
Imports: The necessary libraries are imported for data manipulation (numpy, pandas), machine learning (sklearn), and evaluation metrics.

## ElasticNet: This model is a linear regression algorithm combining L1 (Lasso) and L2 (Ridge) penalties for regularization.

## Target Variable Transformation:

y_train_transformed and y_val_transformed are transformed using np.log1p. This ensures positive predictions by applying a logarithmic transformation, which prevents negatives.

It is often used for skewed data and avoids issues with extreme values

## 2. Pipeline Setup
A Pipeline is created to simplify the process:

StandardScaler: Normalizes features to have zero mean and unit variance. This is crucial for ElasticNet as it is sensitive to feature scaling.

ElasticNet: The regression model included in the pipeline.
## 3. Grid Search for Hyperparameter Tuning
### Parameter Grid: The code defines a set of hyperparameters:

alpha: Controls the overall strength of regularization.

l1_ratio: Balances between Lasso (L1) and Ridge (L2) regularizations.

### GridSearchCV:

Performs an exhaustive search over all hyperparameter combinations using cross-validation (cv=5).

The custom scorer (rmsle_scorer) is used to evaluate models based on the Root Mean Squared Logarithmic Error (RMSLE).
## 4. Custom RMSLE Scorer
### RMSLE Definition: A custom function rmsle is defined:

Clips predictions (y_pred) to non-negative values.

Calculates the square root of the Mean Squared Logarithmic Error (mean_squared_log_error).

### Scorer: The rmsle function is wrapped using make_scorer, ensuring it works seamlessly in GridSearchCV.

## 5. Model Training and Evaluation
Fitting: The GridSearchCV is fit to the transformed training data.

Best Parameters: grid_search.best_params_ outputs the optimal hyperparameter combination.

Best Model: The trained pipeline with the best parameters is stored as best_model.

## 6. Predictions on Validation Set
Predictions are made for the validation data:

Transformations: Apply the inverse transformation (np.expm1) to revert the log scaling and handle negative predictions.

Clipping: Ensures predictions are non-negative and replaces NaNs with zeros.

## 7. RMSLE Evaluation
The RMSLE metric is computed for the validation predictions to evaluate model performance.

Debugging lines print the minimum and maximum predicted values, ensuring predictions are as expected.

## 8. Predictions on Test Data
Similar transformations and predictions are applied to the test set, preparing data for submission.

## 9. Save Predictions
The final predictions (y_test_pred) are saved into a CSV file (submission4.csv), where the target column is assumed to be Rings.

## Conclusions 
This code is robust for regression problems where the target variable has a skewed distribution and non-negative values are required. It uses ElasticNet for modeling, ensures proper scaling, and optimizes hyperparameters using cross-validation


In [73]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_log_error, make_scorer

# Transform target variable using log1p to prevent negative predictions
y_train_transformed = np.log1p(np.maximum(y_train, 0))  # Apply log transform
y_val_transformed = np.log1p(np.maximum(y_val, 0))      # Apply to validation

# Define ElasticNet model
elastic_net = ElasticNet(random_state=42)

# Create a pipeline with StandardScaler and ElasticNet
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('elastic_net', ElasticNet(random_state=42))
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'elastic_net__alpha': [0.01, 0.1, 1, 10],
    'elastic_net__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
}

# Custom RMSLE scorer
def rmsle(y_true, y_pred):
    y_pred = np.maximum(y_pred, 0)  # Clip to ensure non-negative predictions
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

# Initialize GridSearchCV with the custom RMSLE scorer
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=rmsle_scorer)

# Fit the model on the transformed target variable
grid_search.fit(X_train, y_train_transformed)

# Best parameters
print(f"Best parameters: {grid_search.best_params_}")

# Best model
best_model = grid_search.best_estimator_

# Make predictions on the validation set
y_val_pred_transformed = best_model.predict(X_val)

# Convert back using expm1 (inverse of log1p) and clip
y_val_pred = np.expm1(y_val_pred_transformed)
y_val_pred = np.nan_to_num(y_val_pred, nan=0)
y_val_pred[y_val_pred < 0] = 0  # Clip negatives

# Evaluate the model on the validation set using RMSLE
rmsle_val = np.sqrt(mean_squared_log_error(y_val, y_val_pred))
print(f"Best Model RMSLE on Validation Set: {rmsle_val}")

# Make predictions on the test data
y_test_pred_transformed = best_model.predict(X_test)

# Convert back using expm1 and clip
y_test_pred = np.expm1(y_test_pred_transformed)
y_test_pred = np.nan_to_num(y_test_pred, nan=0)
y_test_pred[y_test_pred < 0] = 0  # Clip negatives

# Debugging: Print min/max predictions
print(f"Min y_val_pred: {y_val_pred.min()}, Max y_val_pred: {y_val_pred.max()}")
print(f"Min y_test_pred: {y_test_pred.min()}, Max y_test_pred: {y_test_pred.max()}")

# Save predictions to a submission file
sample_submission["Rings"] = y_test_pred  # Assuming 'Rings' is the target column
sample_submission.to_csv("submission4.csv", index=False)
print("Predictions saved to submission4.csv")


Best parameters: {'elastic_net__alpha': 0.01, 'elastic_net__l1_ratio': 0.1}
Best Model RMSLE on Validation Set: 0.16815280282519535
Min y_val_pred: 3.4677379807765605, Max y_val_pred: 25.901674486722396
Min y_test_pred: 3.3971699771776147, Max y_test_pred: 69.17587241189953
Predictions saved to submission4.csv


# Discussion of Results from Elastic Net Regression
The Abalone dataset is a classic regression problem where the goal is to predict the age of abalones (a type of sea snail) based on physical measurements. The age is typically determined by counting the number of rings in the shell, which is the target variable. 
## Context: Abalone Dataset
Features: Physical measurements like length, diameter, height, weight, etc.

Target Variable: The number of rings (indirectly representing the age of the abalone).

Evaluation Metric: RMSLE (Root Mean Squared Logarithmic Error) is used to evaluate the model's performance. This metric is particularly useful when the target variable (number of rings) has a wide range and you want to penalize errors in predicting smaller values more heavily.

# Interpretation of Results in Context
## 1. Best Parameters: {'elastic_net__alpha': 0.01, 'elastic_net__l1_ratio': 0.1}
The model uses a small regularization strength (alpha = 0.01), which means it is not heavily penalizing large coefficients. This suggests that the features in the dataset are likely contributing meaningfully to the prediction.

The l1_ratio = 0.1 indicates that the model is using mostly L2 (Ridge) regularization, which shrinks coefficients uniformly but does not force them to zero. This is a good choice if most features are expected to be relevant for predicting the number of rings.

## 2. Best Model RMSLE on Validation Set: 0.16815280282519535
The RMSLE score of 0.168 on the validation set indicates how well the model predicts the number of rings. Since RMSLE is in log space, this value suggests that the model's predictions are, on average, reasonably close to the actual number of rings.

For example, if the actual number of rings is 10, the model's prediction would likely fall within the range of e^(log(10) ± 0.168, which is approximately 8.1 to 12.3.

## 3. Min y_val_pred: 3.4677379807765605, Max y_val_pred: 25.901674486722396
These are the minimum and maximum predicted number of rings for the validation set.

The range (3.467 to 25.901) suggests that the model is predicting a wide range of ages, which aligns with the natural variability in the number of rings for abalones.

## 4. Min y_test_pred: 3.3971699771776147, Max y_test_pred: 69.17587241189953
These are the minimum and maximum predicted number of rings for the test set.

The test set predictions have a wider range (3.397 to 69.175) compared to the validation set. This could indicate that the test set contains some abalones with very high numbers of rings (older abalones), which the model is predicting with higher uncertainty.

## 5. Predictions saved to submission4.csv
The predictions for the test set have been saved to submission4.csv. This file contains the predicted number of rings for each abalone in the test set, which can be used for evaluation or submission (e.g., in a competition or as part of a report).

# Key Insights
# Model Performance:

## The RMSLE of 0.168 suggests the model is performing reasonably well, but there is room for improvement. One could could experiment with:

Feature engineering (e.g., creating new features from existing ones).

## Hyperparameter tuning (e.g., trying different values of alpha and l1_ratio).

Using more advanced models (e.g., gradient boosting or neural networks).

## Prediction Range:

The wide range of predictions (3.397 to 69.175) suggests that the model is capturing the variability in the dataset, but the high maximum value (69.175) might indicate some outliers or overestimation for older abalones.Investigations into these cases further are warranted.

## Regularization:
The small alpha value and low l1_ratio suggest that the model is not heavily regularized. If overfitting is a concern,one could try increasing alpha or adjusting the l1_ratio to include more L1 regularization.

# Next Steps
Analyze Feature Importance: Use the coefficients from the Elastic Net model to understand which features are most important for predicting the number of rings.

## Check for Overfitting: Compare the RMSLE on the training set and validation set. If the training error is much lower, the model may be overfitting.

## Experiment with Other Models: Try other regression models (e.g., Random Forest, Gradient Boosting) to see if they perform better on this dataset.
Other regression models on the Abalone data set have been done with these results by this data scientist:
Linear Regression (RMSLE: 0.1668)
The Linear Regression model has the highest RMSLE, indicating that it is the least accurate among the models tested.

This is expected because Linear Regression is a simple model and may not capture complex patterns in the data.

Random Forest (RMSLE: 0.1557)
The Random Forest model performs better than Linear Regression, with a lower RMSLE of 0.1557.

Random Forest is more flexible and can capture non-linear relationships in the data, which explains the improvement.

XGBoost (Default Hyperparameters, RMSLE: 0.1531)
The XGBoost model with default hyperparameters has a lower RMSLE (0.1531) than Random Forest, making it the best-performing model so far.

XGBoost is a powerful ensemble method that often outperforms Random Forest due to its gradient-boosting framework and regularization techniques.

XGBoost (Best Model from Grid Search, RMSLE: 0.1521)
After hyperparameter tuning using Grid Search, the best XGBoost model achieves a slightly lower RMSLE of 0.1521.

This improvement shows that hyperparameter tuning can help optimize the model's performance.

XGBoost (Log-Transformed Target, RMSLE: 0.1514)
Applying a log transformation to the target variable further reduces the RMSLE to 0.1514.

Log transformation helps handle skewness in the target variable, making the model more robust to extreme values.

Clearly per the RMSLE results the Elastic model even with hyperparameter tuning and validation performs the worst with an RMSLE of 0.1682 for the best model. This is even worse than linear regression.

## Investigate Outliers: Look at the abalones with very high predicted rings (e.g., 69.175) to see if they are outliers or if the model is overestimating.|